In [1]:
!pip install -Uqq fastbook

In [2]:
from fastbook import search_images_bing
from fastai.vision.all import *
from pathlib import Path

In [3]:
#DATA COLLECTION

KEY="fedbf36b17484a9cb04296d5ad0e64bc"
path = Path("Heroes")

if not path.exists():
    path.mkdir()
    for hero in ["Marvel", "DC"]:
        sub_path = path/hero
        sub_path.mkdir(exist_ok=True)
        search_results = search_images_bing(KEY, 
                                            f"{hero} Heroes")
        download_images(dest=sub_path, 
                        urls=search_results.attrgot("contentUrl"))

file_paths = get_image_files(path)
bad_images = verify_images(file_paths)
bad_images.map(Path.unlink)

(#0) []

In [4]:
#TRANSFORMATION TO INGESTABLE TENSORS AND FEATURE ENGINEERING

datablock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                      get_items=get_image_files,
                      splitter = RandomSplitter(seed=1),
                      get_y=parent_label,
                      item_tfms=RandomResizedCrop(128, min_scale=0.3),
                      batch_tfms=aug_transforms(mult=2))

dls=datablock.dataloaders(source=path)

learner = cnn_learner(dls, resnet152, metrics=error_rate)
learner.fine_tune()

Downloading: "https://download.pytorch.org/models/resnet152-394f9c45.pth" to /root/.cache/torch/hub/checkpoints/resnet152-394f9c45.pth


  0%|          | 0.00/230M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,1.047300,1.387412,0.377358,00:14


epoch,train_loss,valid_loss,error_rate,time
0,1.143648,0.855620,0.377358,00:14
1,1.009155,0.673035,0.358491,00:15
2,0.965420,0.633912,0.358491,00:14
3,0.929499,0.638031,0.396226,00:13
4,0.903336,0.668154,0.358491,00:13
5,0.871723,0.696211,0.396226,00:14
6,0.863560,0.723106,0.339623,00:14
7,0.830635,0.766337,0.301887,00:13
8,0.822233,0.770355,0.283019,00:14
9,0.806114,0.719725,0.226415,00:14


In [5]:
learner.export("hero_classifier.pkl")

In [5]:
learner_inf = load_learner("hero_classifier.pkl")

In [6]:
from ipywidgets import widgets
from IPython.display import display
import io
from PIL import Image

In [133]:
file_upload = widgets.FileUpload(description="Upload an image",
                         accept=".jpeg,.jpg, .png",
                         multiple=False)
img_out = widgets.Output()
inference = widgets.Label()
run_predict = widgets.Button(description="Predict")

def event(change):
    img_out.clear_output()
    img=Image.open(io.BytesIO(file_upload.data[0]))
    with img_out:
        display(img.to_thumb(250, 250))
    pred, pred_idx, prob = learner_inf.predict(file_upload.data[0])
    inference.value = f"Prediction: {pred}, Probability: {prob[pred_idx]}"
    
run_predict.on_click(event)

widgets.VBox([file_upload, img_out, run_predict, inference])

/opt/conda/lib/python3.7/site-packages/PIL/Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  # after quantization.
